In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import LabelEncoder


In [2]:

def initialize_words():
    return ["Good","Thankyou"]

def capture_frames(word, num_samples=20):
    cap = cv2.VideoCapture(0)
    mp_holistic = mp.solutions.holistic
    mp_drawing = mp.solutions.drawing_utils
    
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    )

    frames = []
    labels = []

    print(f"Capturing {num_samples} frames for '{word}'... Press 'k' to capture a frame.")

    while len(frames) < num_samples:
        ret, frame = cap.read()
        if not ret:
            break

        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        
        results = holistic.process(rgb_frame)
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
        
        
        
        combined_landmarks = np.concatenate([pose, lh, rh])
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        cv2.imshow('Hand Gesture Capture', image)

        
        key = cv2.waitKey(1)
        if key & 0xFF == ord('k'):
            frames.append(combined_landmarks)
            labels.append(word)
            print(f"Captured {len(frames)} frames")

    cap.release()
    cv2.destroyAllWindows()

    return np.array(frames), np.array(labels)

def train_model(X_train, y_train):
    model = keras.Sequential([
        layers.Input(shape=(258,)),  
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(2, activation='softmax')  
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=200, batch_size=32)

    
    # model.save('hand_gesture_model.h5')

    return model

import cv2
import mediapipe as mp
import numpy as np
from tensorflow import keras

def test_model(model,label_encoder):
    cap = cv2.VideoCapture(0)
    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose
    mp_holistic = mp.solutions.holistic
    mp_drawing = mp.solutions.drawing_utils
    
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    )
    hands = mp_hands.Hands()
    pose = mp_pose.Pose()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb_frame)
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    

        input_features = np.concatenate([pose, lh, rh])


        
        prediction = model.predict(np.array([input_features]))
        predicted_class = np.argmax(prediction)
        predicted_word = label_encoder.inverse_transform([predicted_class])[0]

        
        cv2.putText(rgb_frame, predicted_word, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        mp_drawing.draw_landmarks(rgb_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(rgb_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(rgb_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        rgb_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        cv2.imshow("Hand Gesture Recognition", rgb_frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()








In [3]:
words = initialize_words()


all_frames = []
all_labels = []
for word in words:
    frames, labels = capture_frames(word)
    all_frames.append(frames)
    all_labels.append(labels)


X_train = np.concatenate(all_frames)
y_train = np.concatenate(all_labels)

Capturing 20 frames for 'Good'... Press 'k' to capture a frame.
Captured 1 frames
Captured 2 frames
Captured 3 frames
Captured 4 frames
Captured 5 frames
Captured 6 frames
Captured 7 frames
Captured 8 frames
Captured 9 frames
Captured 10 frames
Captured 11 frames
Captured 12 frames
Captured 13 frames
Captured 14 frames
Captured 15 frames
Captured 16 frames
Captured 17 frames
Captured 18 frames
Captured 19 frames
Captured 20 frames
Capturing 20 frames for 'Thankyou'... Press 'k' to capture a frame.
Captured 1 frames
Captured 2 frames
Captured 3 frames
Captured 4 frames
Captured 5 frames
Captured 6 frames
Captured 7 frames
Captured 8 frames
Captured 9 frames
Captured 10 frames
Captured 11 frames
Captured 12 frames
Captured 13 frames
Captured 14 frames
Captured 15 frames
Captured 16 frames
Captured 17 frames
Captured 18 frames
Captured 19 frames
Captured 20 frames


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


# Train the model
model = train_model(X_train, y_train_encoded)

Epoch 1/200
2/2 [==============================] - 1s 8ms/step - loss: 0.7517 - accuracy: 0.4000
Epoch 2/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6554 - accuracy: 0.6500
Epoch 3/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6861 - accuracy: 0.5000
Epoch 4/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6419 - accuracy: 0.6000
Epoch 5/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6351 - accuracy: 0.6000
Epoch 6/200
2/2 [==============================] - 0s 6ms/step - loss: 0.6131 - accuracy: 0.6250
Epoch 7/200
2/2 [==============================] - 0s 5ms/step - loss: 0.5951 - accuracy: 0.8000
Epoch 8/200
2/2 [==============================] - 0s 6ms/step - loss: 0.6373 - accuracy: 0.5000
Epoch 9/200
2/2 [==============================] - 0s 5ms/step - loss: 0.5850 - accuracy: 0.6250
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 0.5951 - accuracy: 0.6000
Epoch 11/200
2/2 [===========

In [5]:
y_train_encoded


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [14]:
# # Test the model in real-time
# from keras.models import load_model
# model = load_model('hand_gesture_model.h5')
# label_encoder=LabelEncoder()
test_model(model,label_encoder)

1/1 [==============================] - 0s 33ms/step


In [6]:
model.save('hand_model.h5')
import joblib

joblib.dump(label_encoder, 'label_encoder.joblib')


c:\Users\Singh\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['label_encoder.joblib']